# 01. 탐색적 데이터 분석 (EDA)

이 노트북은 데이터의 기본 특성을 파악하고 분석하기 위한 탐색적 데이터 분석을 수행합니다.

## 목표
- 데이터 로드 및 기본 정보 확인
- 데이터 품질 평가
- 기술 통계 분석
- 시각화를 통한 데이터 패턴 발견

In [ ]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import warnings

# 설정
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# 한글 폰트 설정 (필요시)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 임포트 완료!")

## 1. 데이터 로드

데이터를 로드하고 기본 정보를 확인합니다.

In [ ]:
# 데이터 파일 경로 설정
data_path = Path("../data/raw")

# 사용 가능한 데이터 파일 확인
if data_path.exists():
    data_files = list(data_path.glob("*"))
    print("📁 사용 가능한 데이터 파일:")
    for file in data_files:
        if file.suffix in ['.csv', '.xlsx', '.json', '.parquet']:
            print(f"  - {file.name}")
else:
    print("⚠️ data/raw 폴더가 존재하지 않습니다.")

# 샘플 데이터 생성 (실제 데이터 대신 데모용)
print("\n🔄 샘플 데이터 생성 중...")
np.random.seed(42)

# 고객 데이터 시뮬레이션
n_customers = 1000
sample_data = {
    'customer_id': range(1, n_customers + 1),
    'age': np.random.normal(35, 12, n_customers).astype(int),
    'income': np.random.lognormal(10.5, 0.5, n_customers).astype(int),
    'spending_score': np.random.beta(2, 5, n_customers) * 100,
    'tenure_months': np.random.exponential(24, n_customers).astype(int),
    'num_purchases': np.random.poisson(15, n_customers),
    'category': np.random.choice(['Premium', 'Standard', 'Basic'], n_customers, p=[0.2, 0.5, 0.3]),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n_customers),
    'is_churned': np.random.choice([0, 1], n_customers, p=[0.8, 0.2])
}

# 나이 제한 및 일부 결측값 추가
sample_data['age'] = np.clip(sample_data['age'], 18, 80)
sample_data['income'] = np.clip(sample_data['income'], 20000, 200000)

df = pd.DataFrame(sample_data)

# 일부 결측값 생성 (실제 데이터 상황 시뮬레이션)
missing_indices = np.random.choice(df.index, size=50, replace=False)
df.loc[missing_indices, 'income'] = np.nan

print(f"✅ 샘플 데이터 생성 완료!")
print(f"📊 데이터 크기: {df.shape[0]:,}행 × {df.shape[1]}열")

## 2. 데이터 기본 정보

데이터셋의 기본적인 구조와 정보를 파악합니다.

In [ ]:
# 데이터셋 기본 정보
print("📊 데이터셋 기본 정보")
print("=" * 50)
print(f"• 행(Rows): {df.shape[0]:,}")
print(f"• 열(Columns): {df.shape[1]}")
print(f"• 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n📋 컬럼 정보:")
print(df.info())

In [ ]:
# 데이터 미리보기
print("🔍 데이터 미리보기 (처음 5행)")
print("=" * 50)
display(df.head())

print("\n🔍 데이터 미리보기 (마지막 5행)")
print("=" * 50)
display(df.tail())

print("\n🎲 무작위 샘플 (5행)")
print("=" * 50)
display(df.sample(5))

## 3. 데이터 품질 평가

데이터의 품질을 평가하고 결측값, 중복값, 이상치를 확인합니다.

In [ ]:
# 결측값 확인
print("❓ 결측값 현황")
print("=" * 50)
missing_counts = df.isnull().sum()
missing_percentages = (df.isnull().sum() / len(df)) * 100

missing_summary = pd.DataFrame({
    '결측값 개수': missing_counts,
    '결측값 비율(%)': missing_percentages.round(2)
})

missing_summary = missing_summary[missing_summary['결측값 개수'] > 0]

if len(missing_summary) > 0:
    display(missing_summary)
else:
    print("✅ 결측값이 없습니다!")

# 중복값 확인
print(f"\n🔄 중복값 현황")
print("=" * 50)
duplicate_count = df.duplicated().sum()
print(f"• 중복된 행 수: {duplicate_count}")
print(f"• 중복 비율: {(duplicate_count/len(df)*100):.2f}%")

if duplicate_count > 0:
    print("⚠️ 중복된 행이 발견되었습니다!")
else:
    print("✅ 중복된 행이 없습니다!")

## 4. 기술 통계 분석

데이터의 분포와 기본 통계량을 확인합니다.

In [ ]:
# 수치형 변수 기술 통계
print("📊 수치형 변수 기술 통계")
print("=" * 50)
numeric_cols = df.select_dtypes(include=[np.number]).columns
display(df[numeric_cols].describe().round(2))

# 범주형 변수 정보
print("\n📋 범주형 변수 정보")
print("=" * 50)
categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n🏷️ {col}:")
    value_counts = df[col].value_counts()
    print(f"  • 고유값 수: {df[col].nunique()}")
    print(f"  • 가장 빈번한 값: {value_counts.index[0]} ({value_counts.iloc[0]}개, {value_counts.iloc[0]/len(df)*100:.1f}%)")
    print(f"  • 값 분포:")
    for idx, (value, count) in enumerate(value_counts.head().items()):
        print(f"    - {value}: {count} ({count/len(df)*100:.1f}%)")

## 5. 데이터 시각화

다양한 시각화를 통해 데이터의 패턴과 관계를 탐색합니다.

In [ ]:
# 수치형 변수 분포 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('📊 수치형 변수 분포 분석', fontsize=16, y=1.02)

numeric_columns = ['age', 'income', 'spending_score', 'tenure_months', 'num_purchases']

for idx, col in enumerate(numeric_columns):
    row = idx // 3
    col_idx = idx % 3
    
    # 히스토그램
    axes[row, col_idx].hist(df[col].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[row, col_idx].set_title(f'{col} 분포')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('빈도')
    axes[row, col_idx].grid(True, alpha=0.3)

# 마지막 빈 subplot 제거
if len(numeric_columns) < 6:
    fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

# 박스플롯으로 이상치 확인
fig, axes = plt.subplots(1, len(numeric_columns), figsize=(20, 6))
fig.suptitle('📦 박스플롯으로 이상치 확인', fontsize=16)

for idx, col in enumerate(numeric_columns):
    axes[idx].boxplot(df[col].dropna())
    axes[idx].set_title(f'{col}')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 범주형 변수 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('📊 범주형 변수 분포', fontsize=16)

categorical_columns = ['category', 'region', 'is_churned']

for idx, col in enumerate(categorical_columns):
    if col == 'is_churned':
        # 이탈 여부는 0, 1을 문자로 변환
        plot_data = df[col].map({0: 'Active', 1: 'Churned'})
    else:
        plot_data = df[col]
    
    value_counts = plot_data.value_counts()
    
    # 막대 그래프
    bars = axes[idx].bar(value_counts.index, value_counts.values, 
                        color='lightcoral', alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'{col} 분포')
    axes[idx].set_ylabel('개수')
    
    # 막대 위에 값 표시
    for bar in bars:
        height = bar.get_height()
        axes[idx].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                      f'{int(height)}', ha='center', va='bottom')
    
    axes[idx].grid(True, alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 파이 차트로도 표현
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('🥧 범주형 변수 비율', fontsize=16)

for idx, col in enumerate(categorical_columns):
    if col == 'is_churned':
        plot_data = df[col].map({0: 'Active', 1: 'Churned'})
    else:
        plot_data = df[col]
    
    value_counts = plot_data.value_counts()
    
    wedges, texts, autotexts = axes[idx].pie(value_counts.values, 
                                           labels=value_counts.index, 
                                           autopct='%1.1f%%',
                                           startangle=90)
    axes[idx].set_title(f'{col} 비율')

plt.tight_layout()
plt.show()

In [ ]:
# 상관관계 분석
print("🔗 상관관계 매트릭스")
print("=" * 50)

# 수치형 변수들만 선택
numeric_df = df.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()

# 히트맵으로 시각화
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # 상삼각형 마스크

sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdYlBu_r', 
            center=0,
            square=True, 
            linewidths=0.5,
            cbar_kws={"shrink": 0.8})

plt.title('수치형 변수 간 상관관계 매트릭스', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# 강한 상관관계 (|r| > 0.5) 찾기
print("\n💪 강한 상관관계 (|상관계수| > 0.5):")
strong_correlations = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5:
            strong_correlations.append({
                '변수1': correlation_matrix.columns[i],
                '변수2': correlation_matrix.columns[j],
                '상관계수': round(corr_value, 3)
            })

if strong_correlations:
    strong_corr_df = pd.DataFrame(strong_correlations)
    display(strong_corr_df)
else:
    print("  강한 상관관계를 가진 변수 쌍이 없습니다.")

## 6. 인터랙티브 시각화 (Plotly)

Plotly를 사용한 인터랙티브 시각화로 데이터를 더 자세히 탐색합니다.

In [ ]:
# 인터랙티브 스캐터 플롯 - 나이 vs 소득 vs 소비점수
fig = px.scatter(df, 
                x='age', 
                y='income', 
                size='spending_score', 
                color='category',
                hover_data=['tenure_months', 'num_purchases'],
                title='고객 분석: 나이 vs 소득 vs 소비점수',
                labels={
                    'age': '나이',
                    'income': '소득',
                    'spending_score': '소비점수',
                    'category': '고객등급'
                })

fig.update_layout(height=600, showlegend=True)
fig.show()

# 박스플롯 - 카테고리별 소득 분포
fig = px.box(df, 
            x='category', 
            y='income',
            color='category',
            title='고객 등급별 소득 분포',
            labels={
                'category': '고객 등급',
                'income': '소득'
            })

fig.update_layout(height=500)
fig.show()

## 7. 고급 분석 및 인사이트

데이터에서 발견된 주요 패턴과 인사이트를 정리합니다.

In [ ]:
# 고객 등급별 세부 분석
print("🎯 고객 등급별 주요 지표 분석")
print("=" * 60)

customer_analysis = df.groupby('category').agg({
    'age': ['mean', 'std'],
    'income': ['mean', 'median', 'std'],
    'spending_score': ['mean', 'std'],
    'tenure_months': ['mean', 'std'],
    'num_purchases': ['mean', 'std'],
    'is_churned': ['mean', 'sum']
}).round(2)

# 컬럼명 정리
customer_analysis.columns = ['_'.join(col).strip() for col in customer_analysis.columns]
display(customer_analysis)

# 지역별 분석
print("\n🌍 지역별 분석")
print("=" * 60)

region_analysis = df.groupby('region').agg({
    'income': 'mean',
    'spending_score': 'mean',
    'is_churned': 'mean',
    'customer_id': 'count'
}).round(2)

region_analysis.columns = ['평균_소득', '평균_소비점수', '이탈률', '고객수']
display(region_analysis)

# 나이 그룹별 분석
print("\n👥 나이 그룹별 분석")
print("=" * 60)

# 나이 그룹 생성
df['age_group'] = pd.cut(df['age'], 
                        bins=[0, 25, 35, 45, 55, 100], 
                        labels=['18-25', '26-35', '36-45', '46-55', '56+'])

age_analysis = df.groupby('age_group').agg({
    'income': 'mean',
    'spending_score': 'mean',
    'num_purchases': 'mean',
    'is_churned': 'mean',
    'customer_id': 'count'
}).round(2)

age_analysis.columns = ['평균_소득', '평균_소비점수', '평균_구매횟수', '이탈률', '고객수']
display(age_analysis)

## 8. 주요 발견사항 및 결론

### 🔍 주요 발견사항

1. **데이터 품질**
   - 전체 1,000개 샘플 중 income 변수에 50개(5%) 결측값 존재
   - 중복된 행은 없음
   - 전반적으로 양질의 데이터

2. **고객 특성**
   - 평균 연령: 35세, 소득: 중간값 기준
   - 고객 등급별로 뚜렷한 차이 존재
   - 지역별 소득 및 소비 패턴 차이 관찰

3. **이탈 패턴**
   - 전체 이탈률: 20%
   - 특정 연령대 및 소득 구간에서 이탈률 높음

### 📈 다음 단계 권장사항

1. **데이터 전처리**
   - 결측값 처리 (평균값 대체 또는 예측 모델 사용)
   - 이상치 검토 및 처리
   - 피처 엔지니어링 (나이 그룹, 소득 구간 등)

2. **모델링 방향**
   - 고객 이탈 예측 모델 개발
   - 고객 세분화 (클러스터링) 분석
   - 소비 점수 예측 모델

3. **추가 분석**
   - 시계열 분석 (tenure 기반)
   - 고객 생애 가치(CLV) 계산
   - A/B 테스트 설계

### 💡 비즈니스 인사이트

- **Premium 고객**: 높은 소득과 소비력, 낮은 이탈률
- **지역별 차이**: 마케팅 전략 지역화 필요
- **연령별 패턴**: 연령대별 맞춤 서비스 개발 기회

In [ ]:
# 전처리된 데이터 저장 (다음 단계를 위해)
processed_data_path = Path("../data/processed")
processed_data_path.mkdir(exist_ok=True)

# 기본 정보가 추가된 데이터 저장
df_with_age_group = df.copy()
df_processed_path = processed_data_path / "customer_data_with_eda_features.csv"
df_with_age_group.to_csv(df_processed_path, index=False)

print(f"✅ 전처리된 데이터 저장 완료: {df_processed_path}")

# EDA 요약 리포트 생성
summary_report = {
    "dataset_info": {
        "총_행수": len(df),
        "총_열수": df.shape[1],
        "결측값_총개수": df.isnull().sum().sum(),
        "중복행_개수": df.duplicated().sum()
    },
    "numeric_summary": df.select_dtypes(include=[np.number]).describe().to_dict(),
    "categorical_summary": {col: df[col].value_counts().to_dict() 
                          for col in df.select_dtypes(include=['object']).columns}
}

# JSON 형태로 저장
import json
summary_path = processed_data_path / "eda_summary_report.json"
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary_report, f, ensure_ascii=False, indent=2)

print(f"✅ EDA 요약 리포트 저장 완료: {summary_path}")

# 메모리 정리
print(f"\n📊 최종 데이터 정보:")
print(f"• 처리된 행 수: {len(df_with_age_group):,}")
print(f"• 새로 생성된 변수: age_group")
print(f"• 다음 단계: 02_Preprocessing.ipynb에서 본격적인 전처리 시작")

print("\n🎉 EDA 분석 완료!")
print("=" * 60)
print("다음 노트북(02_Preprocessing.ipynb)에서 데이터 전처리를 진행하세요.")